In [1]:
pip install -U scikit-learn==1.7.1 


  Using cached scikit_learn-1.7.1-cp311-cp311-win_amd64.whl.metadata (11 kB)
Using cached scikit_learn-1.7.1-cp311-cp311-win_amd64.whl (8.9 MB)
   ---------------------------------------- 0.0/38.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/38.7 MB ? eta -:--:--
    --------------------------------------- 0.5/38.7 MB 932.9 kB/s eta 0:00:41
    --------------------------------------- 0.8/38.7 MB 958.5 kB/s eta 0:00:40
    --------------------------------------- 0.8/38.7 MB 958.5 kB/s eta 0:00:40
   - -------------------------------------- 1.0/38.7 MB 967.3 kB/s eta 0:00:39
   - -------------------------------------- 1.3/38.7 MB 986.4 kB/s eta 0:00:38
   - -------------------------------------- 1.6/38.7 MB 987.0 kB/s eta 0:00:38
   - -------------------------------------- 1.8/38.7 MB 996.7 kB/s eta 0:00:38
   - -------------------------------------- 1.8/38.7 MB 996.7 kB/s eta 0:00:38
   -- --------

In [2]:
pip install numpy pandas nltk rouge-score transformers joblib matplotlib


  Using cached rouge_score-0.1.2-py3-none-any.whl
  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached click-8.3.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached pyyaml-6.0.3-cp311-cp311-win_amd64.whl.metadata (2.4 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.6.2-cp38-abi3-win_amd64.whl.metadata (4.1 kB)
  Using cached charset_normalizer-3.4.4-cp311-cp311-win_amd64.whl.metadata (38 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.10.5-py3-none-any.whl.metadata (2.5 kB)
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.3 

In [4]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())


2.9.0+cpu
False


In [5]:
# 📊 Email Assistant – Real Model Evaluation Notebook
# Author: Magna Benita
# Repository: email-assistant

# --------------------------------------------------------
# This notebook evaluates the performance of each module:
# 1. Intent Recognition
# 2. Paraphrase Generation (T5)
# 3. Politeness Scoring
# 4. Template Recommendation
# 5. End-to-End Latency
# --------------------------------------------------------

import time
import numpy as np
import pandas as pd
import joblib
import json
import random
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import matplotlib.pyplot as plt




c:\Users\PRABHAKARAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Load datasets
emails_df = pd.read_csv("enron_dataset/enron_raw.csv")
templates = json.load(open("enron_dataset/email_templates.json"))

##  1. Intent Recognition Evaluation
We evaluate the TF-IDF + classifier pipeline for intent recognition using a held-out Enron subset.


In [7]:
# Load vectorizer and corpus model
vectorizer = joblib.load("models/tfidf_vectorizer.pkl")
intent_model = joblib.load("models/email_corpus.pkl")

# Select small random sample for evaluation
sample_df = emails_df.sample(50, random_state=42)
X_test = vectorizer.transform(sample_df["text"])
y_true = sample_df["intent"]

# Predict intents
y_pred = intent_model.predict(X_test)

intent_acc = accuracy_score(y_true, y_pred)
print(f"Intent Recognition Accuracy: {intent_acc * 100:.2f}%")


KeyError: 'text'

##  2. Paraphrase Generation Evaluation
We use the "Vamsi/T5_Paraphrase_Paws" model to generate paraphrases and compute BLEU / ROUGE-L.


In [8]:
tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")
model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws")

rouge = Rouge()
bleu_scores, rouge_scores = [], []

samples = sample_df["text"].head(5).tolist()

for sent in samples:
    input_text = "paraphrase: " + sent + " </s>"
    encoding = tokenizer.encode_plus(input_text, padding=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**encoding, max_length=60, num_beams=4, temperature=1.5)
    paraphrase = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    bleu = sentence_bleu([sent.split()], paraphrase.split())
    rouge_l = rouge.get_scores(paraphrase, sent)[0]['rouge-l']['f']
    bleu_scores.append(bleu)
    rouge_scores.append(rouge_l)

bleu_avg = np.mean(bleu_scores)
rouge_avg = np.mean(rouge_scores)

print(f"BLEU: {bleu_avg:.2f}")
print(f"ROUGE-L: {rouge_avg:.2f}")


ImportError: 
 requires the protobuf library but it was not found in your environment. Check out the instructions on the
installation page of its repo: https://github.com/protocolbuffers/protobuf/tree/master/python#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


##  3. Politeness Scoring Evaluation
Since politeness is human-perceived, we use a heuristic function based on presence of polite words.


In [9]:
polite_words = ["please", "thank you", "could you", "kindly", "appreciate", "would you"]
def politeness_score(text):
    text_lower = text.lower()
    score = sum(1 for w in polite_words if w in text_lower) / len(polite_words)
    return round(3 + 2 * score, 2)  # scale 1–5

sample_df["politeness"] = sample_df["text"].apply(politeness_score)
avg_politeness = sample_df["politeness"].mean()
print(f"Average Politeness Rating: {avg_politeness:.1f} / 5")


KeyError: 'text'

##  4. Template Recommendation Evaluation
We measure recommendation performance using Average Precision at 5 (AP@5)
based on cosine similarity between input email and template corpus.


In [ ]:
template_texts = [t["template_text"] for t in templates]
template_vecs = vectorizer.transform(template_texts)

ap_values = []

for _, row in sample_df.head(10).iterrows():
    query_vec = vectorizer.transform([row["text"]])
    sims = cosine_similarity(query_vec, template_vecs).flatten()
    top5_idx = sims.argsort()[-5:][::-1]
    
    # Assume relevance = 1 if same intent label
    relevant = [1 if templates[i]["intent"] == row["intent"] else 0 for i in top5_idx]
    ap5 = np.mean(relevant)
    ap_values.append(ap5)

ap5_score = np.mean(ap_values)
print(f"AP@5: {ap5_score:.2f}")


##  5. End-to-End Latency Evaluation
We measure time taken from input → intent → paraphrase → recommendation.


In [ ]:
latencies = []

for _, row in sample_df.head(10).iterrows():
    start = time.time()
    
    # Intent
    _ = intent_model.predict(vectorizer.transform([row["text"]]))
    
    # Paraphrase
    input_text = "paraphrase: " + row["text"] + " </s>"
    enc = tokenizer.encode_plus(input_text, return_tensors="pt", padding=True)
    _ = model.generate(**enc, max_length=50, num_beams=2)
    
    # Recommendation
    query_vec = vectorizer.transform([row["text"]])
    _ = cosine_similarity(query_vec, template_vecs).flatten()
    
    latencies.append(time.time() - start)

avg_latency = np.mean(latencies)
print(f"Average Response Time: {avg_latency:.2f} seconds")

plt.plot(latencies)
plt.title("End-to-End Response Time per Request")
plt.xlabel("Request #")
plt.ylabel("Seconds")
plt.show()
